In [1]:
'''!pip install google-api-python-client'''

'!pip install google-api-python-client'

In [2]:
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials
from googleapiclient.http import MediaFileUpload

In [3]:
SCOPES = ['https://www.googleapis.com/auth/drive']
CREDENTIALS_FILE = 'C:\\Users\\yanme\\OneDrive\\Área de Trabalho\\Portifólio\\yfanalise\\yfinance-data-analisys-628f390e0a95.json'
credentials = Credentials.from_service_account_file(CREDENTIALS_FILE, scopes=SCOPES)
drive = build('drive', 'v3', credentials=credentials)
drive_service = build('drive', 'v3', credentials=credentials)

######################### Funções
def delete_file(file_name, folder_id):
    try:
        # Listar todos os arquivos na pasta de destino
        response = drive_service.files().list(q=f"'{folder_id}' in parents",
                                               fields="files(id, name)").execute()
        files = response.get('files', [])
        
        # Procurar pelo arquivo com o nome específico
        file_id = None
        for file in files:
            if file['name'] == file_name:
                file_id = file['id']
                break
        
        # Se o arquivo for encontrado, exclua-o
        if file_id:
            drive_service.files().delete(fileId=file_id).execute()
            print(f"Arquivo '{file_name}' excluído com sucesso.")
        else:
            print(f"Arquivo '{file_name}' não encontrado na pasta.")
    except Exception as e:
        print(f"Erro ao excluir arquivo: {e}")

def upload_file(file_path, file_name, folder_id):
    # Faz upload do arquivo para o Google Drive
    media_body = MediaFileUpload(file_path, mimetype='application/octet-stream')
    file_metadata = {
        'name': file_name,
        'parents': [folder_id]
    }
    drive_service.files().create(
        body=file_metadata,
        media_body=media_body,
        fields='id'
    ).execute()
    print(f"Upload do arquivo '{file_name}' concluído com sucesso.")

In [6]:
name = 'yfhistoric.db'
file_path = f'C:\\Users\\yanme\\OneDrive\\Área de Trabalho\\Portifólio\\yfanalise\\yfhistoric.db'
folder_id = '1sX6i8z3Oo7q9kCgfiZbb7jYegCA54LrL'

# Excluir arquivo existente, se houver
delete_file(name, folder_id)
# Fazer upload do novo arquivo
upload_file(file_path, name, folder_id)

Arquivo 'yfhistoric.db' excluído com sucesso.
Upload do arquivo 'yfhistoric.db' concluído com sucesso.


In [11]:
def get_subfolders(folder_id):
    subfolders = []
    try:
        # Listar todos os arquivos na pasta especificada
        response = drive_service.files().list(q=f"'{folder_id}' in parents and mimeType='application/vnd.google-apps.folder'",
                                               fields="files(id, name)").execute()
        files = response.get('files', [])
        
        # Extrair IDs e nomes das subpastas
        subfolders = [(file['id'], file['name']) for file in files]
    except Exception as e:
        print(f"Erro ao obter subpastas: {e}")
    
    return subfolders

# Exemplo de uso:
subfolders = get_subfolders(folder_id)
print("IDs e nomes das subpastas:")
for subfolder_id, subfolder_name in subfolders:
    print(f"ID: {subfolder_id}, Nome: {subfolder_name}")

IDs e nomes das subpastas:
ID: 1WM_KG0gzV041sxLt3038_hCKkk-OKB49, Nome: test2
ID: 1Pjkgdd5KjQv4U--YQZyjPNQgK4EVV-cj, Nome: test


In [ ]:
def backup(